# Harmonizome ETL: Cancer Cell Line Encyclopedia (CCLE)

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Source: https://portals.broadinstitute.org/ccle/data

In [ ]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [ ]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [ ]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

# Initialization

In [ ]:
%%appyter hide_code

{% do SectionField(
    name='data',
    title='Upload Data',
    img='load_icon.png'
) %}

In [ ]:
%%appyter code_eval

{% do DescriptionField(
    name='description',
    text='The examples below were sourced from <a href="https://portals.broadinstitute.org/ccle/data" target="_blank">portals.broadinstitute.org</a>. The downloads require a login so clicking on the examples may not work, in which case they should be downloaded directly from the source website.',
    section='data'
) %}

{% set matrix_file = FileField(
    constraint='.*\.gz$',
    name='expression_matrix', 
    label='RNA-Seq Expression Matrix (gct.gz)', 
    default='Input/CCLE/CCLE_RNAseq_genes_counts_20180929.gct.gz',
    examples={
        'CCLE_RNAseq_genes_counts_20180929.gct.gz': 'https://data.broadinstitute.org/ccle/CCLE_RNAseq_genes_counts_20180929.gct.gz'
    },
    section='data')
%}

{% set sample_file= FileField(
    constraint='.*\.txt$',
    name='cell_annot', 
    label='Cell Line Annotations (txt)', 
    default='Input/CCLE/Cell_lines_annotations_20181226.txt',
    examples={
        'Cell_lines_annotations_20181226.txt': 'https://data.broadinstitute.org/ccle/Cell_lines_annotations_20181226.txt'
    },
    section='data')
%}

### Load Mapping Dictionaries

In [ ]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

### Output Path

In [ ]:
output_name = 'ccle'

path = 'Output/CCLE'
if not os.path.exists(path):
    os.makedirs(path)

# Load Data

In [ ]:
%%appyter code_exec

matrix = pd.read_csv(
    {{matrix_file}}, 
    sep='\t', index_col=0, skiprows=2,
    usecols=lambda c: c != 'Name')

In [ ]:
matrix.head()

In [ ]:
matrix.shape

## Load Sample Metadata

In [ ]:
%%appyter code_exec

sample_meta = pd.read_csv(
    {{sample_file}}, 
    sep='\t', usecols=['CCLE_ID', 'Name', 'Gender', 'Site_Primary', 'Histology'],
    index_col=1)

In [ ]:
sample_meta.head()

In [ ]:
sample_meta.shape

# Pre-process Data

## Map CCLE ID to Cell Line Name

In [ ]:
matrix = matrix.rename(columns=dict(zip(
    sample_meta['CCLE_ID'], sample_meta.index)))
matrix.index.name = 'Gene Symbol'
matrix.columns.name = 'Cell Line'
matrix.head()

In [ ]:
matrix.shape

## Drop Missing Data from Sample Metadata

In [ ]:
sample_meta = sample_meta.reset_index().dropna(subset=['Name']).set_index('Name')
sample_meta.head()

## Save Unfiltered Matrix to file

In [ ]:
uf.save_data(matrix, path, output_name + '_matrix_unfiltered',
            compression='gzip', dtype=np.float32)

# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [ ]:
matrix = uf.map_symbols(matrix, symbol_lookup)
matrix.shape

## Merge Duplicate Genes By Rows and Duplicate Columns

In [ ]:
matrix = uf.merge(matrix, 'row')
matrix = uf.merge(matrix, 'column')
matrix.shape

## Remove Data that is More Than 95% Missing and Impute Missing Data

In [ ]:
matrix = uf.remove_impute(matrix)
matrix.head()

In [ ]:
matrix.shape

## Log2 Transform

In [ ]:
matrix = uf.log2(matrix)
matrix.head()

## Normalize Matrix (Quantile Normalize the Matrix by Column)

In [ ]:
matrix = uf.quantile_normalize(matrix)
matrix.head()

## Normalize Matrix (Z-Score the Rows)

In [ ]:
matrix = uf.zscore(matrix)
matrix.head()

## Histogram of First Sample

In [ ]:
matrix.iloc[:, 0].hist(bins=100)

## Histogram of First Gene

In [ ]:
matrix.iloc[0, :].hist(bins=100)

## Save Filtered Matrix

In [ ]:
uf.save_data(matrix, path, output_name + '_matrix_filtered', 
            ext='tsv', compression='gzip')

# Analyze Data

## Create Gene List

In [ ]:
gene_list = uf.gene_list(matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(matrix, sample_meta)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create matrix of Standardized values (values between -1, and 1)

In [ ]:
standard_matrix = uf.standardized_matrix(matrix)
standard_matrix.head()

In [ ]:
uf.save_data(standard_matrix, path, output_name + '_standard_matrix',
            ext='tsv', compression='gzip')

## Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [ ]:
plt.plot(matrix[matrix.columns[0]],
         standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

## Create Ternary Matrix

In [ ]:
ternary_matrix = uf.ternary_matrix(standard_matrix)
ternary_matrix.head()

In [ ]:
uf.save_data(ternary_matrix, path, output_name + '_ternary_matrix',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(ternary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(ternary_matrix, 'gene', 'down', path, output_name + '_gene_down_set')

In [ ]:
uf.save_setlib(ternary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

In [ ]:
uf.save_setlib(ternary_matrix, 'attribute', 'down', path, 
                             output_name + '_attribute_down_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(standard_matrix.T, 'cosine')
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(standard_matrix, 'cosine')
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf.edge_list(standard_matrix)
uf.save_data(edge_list, path, output_name + '_edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)